In [85]:
import sys
import os
import numpy as np
import requests
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))

import time
import talib
import sqlite3
import datetime
import pandas as pd
from util.util import get_ucodes, get_random_header

path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')

In [130]:
map = {
    '上證_90s': 'SSEC',
    '上證50': '000016',
    '新华富时A50指数': 'xin9x000'
}

for k, v in map.items():
    df = pd.read_csv(f'{k}.csv')
    df['Date']= pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
    df = df.sort_values(by='Date', ascending=True)
    df.columns = ['stime', 'open', 'high', 'low', 'close']
    if v == 'SSEC':
        df['code'] = '.'+v
    else:
        df['code'] = v
    df['volume'] = 0
    conn = sqlite3.connect(path_sqlite)
    tb_name = 's_' + v.lower()

    # conn.execute("DROP TABLE %s" % tb_name)
    conn.execute("CREATE TABLE IF NOT EXISTS %s (stime TEXT PRIMARY KEY, code TEXT NOT NULL, close REAL, "
                 "high REAL, low REAL, open REAL, volume REAL)" % tb_name)
    
    stmt = f"REPLACE INTO {tb_name} (stime, open, high, low, close, code, volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
    conn.executemany(stmt, df.values)
    
    conn.commit()
    conn.close()

In [3]:
# https://www.chiefgroup.com.hk/hk/funds/fundsinfo?rl=Funds_Shortcut&secID=F0000147JI
mapping = {'華夏精選固定收益配置基金 - A類累積 - 港幣': 'chsfiih'}
for k, v in mapping.items():
    df = pd.read_csv(f'{k}.csv')
    df.columns = ['stime', 'close']
    df['code'] = v
    df['open'] = np.nan
    df['high'] = np.nan
    df['low'] = np.nan
    df['volume'] = np.nan
    
    conn = sqlite3.connect(path_sqlite)
    tb_name = 's_' + v.lower().replace('.', '').replace('ss', '').replace('sz', '')

    # conn.execute("DROP TABLE IF EXISTS %s" % tb_name)
    conn.execute("CREATE TABLE IF NOT EXISTS %s (stime TEXT PRIMARY KEY, code TEXT NOT NULL, close REAL, "
                 "high REAL, low REAL, open REAL, volume REAL)" % tb_name)
    
    stmt = f"INSERT OR IGNORE INTO {tb_name} (stime, close, code, open, high, low, volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
    conn.executemany(stmt, df.values)
    
    conn.commit()
    conn.close()

In [115]:
mapping = {
    'chasaau': f'https://api.ifund.com.hk/stock-service/v1/funds/0/chart?fundCode=130008&type=FIVE_YEARS&lang=zh-HK',
    'chsfiih': f'https://api.ifund.com.hk/stock-service/v1/funds/0/chart?fundCode=130006&type=FIVE_YEARS&lang=zh-HK',
    'efedfaa': f'https://api.ifund.com.hk/stock-service/v1/funds/0/chart?fundCode=291009&type=FIVE_YEARS&lang=zh-HK',
    'cuischi': f'https://api.ifund.com.hk/stock-service/v1/funds/0/chart?fundCode=142002&type=FIVE_YEARS&lang=zh-HK'
}

for ucode, v in mapping.items():
    r=requests.get(v)
    data3 = r.json()
    
    name = data3['fundName'].replace("'", "")
    print(name)
    
    df3 = pd.DataFrame(data=data3['points'])
    df3.columns = ['stime', 'close']
    for k, v in df3.iterrows():
        df3.loc[k, 'stime'] = datetime.datetime.utcfromtimestamp(v['stime']).strftime("%Y-%m-%d")
    df3['code'] = ucode
    df3['open'] = np.nan
    df3['high'] = np.nan
    df3['low'] = np.nan
    df3['volume'] = np.nan
    
    conn = sqlite3.connect(path_sqlite)
    tb_name = 's_' + ucode.lower().replace('.', '').replace('ss', '').replace('sz', '')
    
    cur_time = datetime.datetime.now().strftime("%Y-%m-%d")
    conn.execute(f"INSERT OR IGNORE INTO name (code, nmll, stime) VALUES('{ucode}', '{name}', '{cur_time}')")

    conn.execute("CREATE TABLE IF NOT EXISTS %s (stime TEXT PRIMARY KEY, code TEXT NOT NULL, close REAL, "
                 "high REAL, low REAL, open REAL, volume REAL)" % tb_name)
    
    stmt = f"INSERT OR IGNORE INTO {tb_name} (stime, close, code, open, high, low, volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
    conn.executemany(stmt, df3.values)
    
    conn.commit()
    conn.close()
    time.sleep(10)

華夏精選亞洲債券基金A USD DIST
華夏精選固定收益配置基金A HKD Acc
易方達精選策略系列 - 易方達中國股票股息基金 A Acc 港元
匯添富中港策略基金 A類港元
